In [ ]:
import datetime
import pandas as pd
import numpy as np
import os
import pathlib
import email.message

from email.mime.base import MIMEBase
from email.mime.application import MIMEApplication
from apscheduler.executors.pool import ThreadPoolExecutor
from apscheduler.schedulers.background import BackgroundScheduler

executors = {'default': ThreadPoolExecutor(5)}
job_defaults = {'coalesce': False, 'max_instances': 3}

scheduler = BackgroundScheduler(executors=executors, job_defaults=job_defaults)

hour_start = hh
minute_start = mm



In [ ]:
def my_func():
    
    # Последний и предпоследний созданные файлы
    path_to_folder = (r'***')  # директория где лежат нужные таблицы
    list_of_files = os.listdir(path_to_folder)
    latest_file = list_of_files[-2]
    previous_file = list_of_files[-3]
    
    # Загрузка двух последних df, фикс параметров
    
    df1 = pd.read_csv(path_to_folder + '\\' + previous_file)
    df1 = df1.dropna(subset=['col_1']) 
    df1 = df1.loc[df1['col_10'] == 'value_1']
    df1 = df1.loc[df1['col_11'] == 'value_2']
    df1 = df1.loc[df1['col_9'] == 'value_3']
    df1 = df1.loc[df1['col_7'] == 'value_4']
    
    
    
    df2 = pd.read_csv(path_to_folder + '\\' + latest_file)
    df2 = df2.dropna(subset=['col_1']) 
    df2 = df2.loc[df2['col_10'] == 'value_1']
    df2 = df2.loc[df2['col_11'] == 'value_2']
    df2 = df2.loc[df2['col_9'] == 'value_3']
    df2 = df2.loc[df2['col_7'] == 'value_4']
    
    # Идентификатор для сравнения
    
    df1['unique_id'] = (df1['col_1'] + df1['col_2'] + df1['col_3'].astype('str') +
       df1['col_4'].astype('str') + df1['col_5'].astype('str') + df1['col_6'].astype('str') +
       df1['col_7'].astype('str') + df1['col_8'].astype('str') + 
       df1['col_9'].astype('str') + df1['col_10'] + df1['col_11'].notna().astype('str'))
    
    df2['unique_id'] = (df2['col_1'] + df2['col_2'] + df2['col_3'].astype('str') + 
       df2['col_4'].astype('str') + df2['col_5'].astype('str') + df2['col_6'].astype('str') +
       df2['col_7'].astype('str') + df2['col_8'].astype('str') + 
       df2['col_9'].astype('str') + df2['col_10'] + df2['col_11'].notna().astype('str'))
    
    # Показатель считается только по общим идентификаторам (Если добавились новые, то мы их игнорируем, так как раньше их не было) 
    
    df2 = df2.loc[df2['unique_id'].isin(df1['unique_id'].values)]
    df1 = df1.loc[df1['unique_id'].isin(df2['unique_id'].values)]

    df1 = df1.set_index('unique_id')
    df2 = df2.set_index('unique_id')

    df2['совпали id'] = np.where(df2.index == df1.index, 1, 0)
    
    # Расчет показателя(было-стало/было)
    
    df2['diff'] = np.where(df2['совпали id'] == 1, 
                      (df2['col_12'] - df1['col_12']) / df1['col_12'],
                       0)
    
    # Словарь для mapping
    
    'col_5_dict' = {
                   ***
    }
    
    # Переименовать столбцы
    
    df2['col_5'].astype('str')
    df2['col_5'] = df2['col_5'].map('col_5_dict')
    
    # Сводная таблица
    
    pivot = pd.pivot_table(df2,
               index = (Столбцы для синхронизации),
               columns = 'col_5',
               values = 'diff',
               aggfunc = np.sum
               )
    
    # Меняем порядок столбцов, чтобы было удобнее
    
    pivot_export=pivot[
                   [Названия столбцов]
    ]
    
    # Удаляет столбцы, в которых не было изменений
    
    pivot_export_compact = pivot_export.replace(to_replace={0: np.NaN}, value=None, method=None)\
                       .dropna(axis=0, how='all').dropna(axis=1, how='all')

    
    # удаляем стобцы, у которых зафиксировали значение
    # сбрасываем индексы

    pivot_export = pivot_export.reset_index().drop(['col_7', 'col_8', 'col_9', 'col_10', 'col_11'], axis = 1)
    pivot_export_compact = pivot_export_compact.reset_index().drop(['col_7', 'col_8', 'col_9', 'col_10', 'col_11'], axis = 1)
    
    # Покраска значений
    
    def format_text(df, name):
        writer = pd.ExcelWriter(r'path_to_store' + '\\' + name)
        df.to_excel(writer, index=False, sheet_name = 'Sheet1')
        for column in df:
            column_width = max(df[column].astype(str).map(len).max(), len(column)) + 3
            col_idx = df.columns.get_loc(column)
            writer.sheets['Sheet1'].set_column(col_idx, col_idx, column_width)

        workbook = writer.book
        worksheet = writer.sheets['Sheet1']
        format_red = workbook.add_format({'bg_color': '#FFC7CE',
                                          'font_color': '#9C0006',
                                          'num_format': '0.00%'})

        format_green = workbook.add_format({'bg_color':'#C6EFCE',
                                            'font_color': '#006100',
                                            'num_format': '0.00%'})

        format_percent = workbook.add_format({'num_format': '0.00%'})
        worksheet.set_column(4, df.shape[1], None, format_percent)

        (max_row, max_col) = df.shape

        worksheet.conditional_format(1, 4, max_row, max_col-1, {'type':'cell',
                                                                  'criteria': '>',
                                                                  'value': 0,
                                                                  'format':format_green})

        worksheet.conditional_format(1, 4, max_row, max_col-1, {'type':'cell',
                                                                  'criteria': '<',
                                                                  'value': 0,
                                                                  'format':format_red})

        writer.save()
    
    # Создание папки для хранения результатов
    
    if not os.path.isdir('path_to_store'):
        os.mkdir('path_to_store')
        
    # Сохранение в excel
    
    files = [] # Список для прикрепления файлов
    
    
    itog = str(datetime.date.today()) + '_file_name' + '.xlsx'
    itog_compact = str(datetime.date.today()) + '_file_name' + '_compact' + '.xlsx'
    if pivot_export_compact.shape[1] == 4:
        subject = "Тема от " + str(datetime.date.today())
        body = 'Добрый день, коллеги. \
          \nКакой-то текст\
          \n \
          \nАвтоматическая рассылка'
    else:
        result = pivot_export
        result_compact = pivot_export_compact
        format_text(result,itog)
        format_text(result_compact, itog_compact)
            
    # Подготовка к отправке
    
        names = (itog, itog_compact)
        for name in names:
            file = open((r'path_to_store' + '\\' + name), 'rb').read()
            attachment = MIMEApplication(file)
            attachment.add_header('Content-Disposition', 'attachment; filename="%s"' % name)
            files.append(attachment)
        subject = "Тема от " + str(datetime.date.today())
        body = ('Добрый день, коллеги. \
                \nПрикладываю изменения' 
                + '\nНа основе файлов от :' + 
                '\n' + 10*' '+str(latest_file)[:4] +'/' +str(latest_file)[4:6] + '/' + str(latest_file)[6:8] + 
                '\n' + 10*' '+str(previous_file)[:4] +'/' +str(previous_file)[4:6] + '/' + str(previous_file)[6:8] + 
                '\n \
                \nАвтоматическая рассылка.')
        
    f = open('reciever_email_list.txt', 'r')
    reciever_email_list = list(map(str, f.read().split()))
    f.close()
    
    # Функция рассылки
    
    def send_email(reciever_email_list, subject, body, sender_email=None, attachment_list=None):
    
        success=False

        import smtplib
        import ssl
        from email import encoders
        from email.mime.base import MIMEBase
        from email.mime.multipart import MIMEMultipart
        from email.mime.text import MIMEText
        from cryptography.fernet import Fernet

        f = open('crypto.key', 'rb')
        key = f.read()
        f.close()

        fer = Fernet(key)

        f = open('login.txt', 'rb')
        login = fer.decrypt(f.read()).decode()
        f.close()

        login_email = login+"@***.ru" # .com

        if(sender_email==None):
            sender_email=login_email

        f = open('password.txt', 'rb')
        pwd = fer.decrypt(f.read()).decode()
        f.close()

        # Create a multipart message and set headers
        message = MIMEMultipart()
        message["From"] = sender_email
        message["To"] = ", ".join(reciever_email_list)
        message["Subject"] = subject
        message["Bcc"] = ", ".join(reciever_email_list)  # Recommended for mass emails

        # Add body to email
        message.attach(MIMEText(body, "plain"))

        # Add attachment to message and convert message to string
        if(attachment_list!=None):
            for attachment in attachment_list:
                message.attach(attachment)

        text = message.as_string()

        # Log in to server using secure context and send email
        smtp_server = "***"
        port = ***  # For starttls

        server = smtplib.SMTP(smtp_server,port)
        context = ssl.create_default_context()
        server.connect(smtp_server, port)
        server.ehlo()
        try:
            server.login(login_email, pwd)
            server.sendmail(sender_email, reciever_email_list, text)

            success = True
        except:
            f = open('send_email_error.txt','w')
            traceback.print_exc(file = f)
            f.close()
        server.close()

        return success
    
    
    send_email(reciever_email_list, subject, body, sender_email=None, attachment_list=files)

In [ ]:
my_job = scheduler.add_job(my_func, 'cron', hour=hour_start, minute=minute_start, misfire_grace_time=60)

scheduler.start()



In [ ]:
#my_func()

In [ ]:
#my_job.remove()